## Install Selenium

In [ ]:
import importlib
try:
    importlib.import_module('selenium')
    print('selenium is already installed')
except:
    !pip install scrapy
    print('selenium is now installed')
try:
    importlib.import_module('anticaptchaofficial')
    print('anticaptchaofficial is already installed')
except:
    !pip install anticaptchaofficial
    print('anticaptchaofficial is now installed')
try:
    importlib.import_module('webdriver_manager')
    print('webdriver-manager is already installed')
except:
    !pip install webdriver-manager
    print('webdriver-manager is now installed')
try:
    importlib.import_module('speech_recognition')
    print('Speech Recognition is already installed')
except:
    !pip install SpeechRecognition
    print('Speech Recognition is now installed')
try:
    importlib.import_module('fake_useragent')
    print('fake_useragent is already installed')
except:
    !pip install fake-useragent
    print('fake_useragent is now installed')

# Import Necessary Modules

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException
import speech_recognition as sr
from random import randint
from time import sleep

## Initialize the webdriver

In [ ]:
options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
ser = Service('/home/irfan/Downloads/chromedriver_linux64')
driver = webdriver.Chrome(chrome_options=options, service=ser)
driver.maximize_window()

## Open the Superkicks website

In [ ]:
driver.get("https://www.superkicks.in/account/login")

## Wait for the page to load

In [ ]:
wait = WebDriverWait(driver, 10)
header = wait.until(EC.presence_of_element_located((By.ID, 'MegaMenu-Content-2')))

# Login the Acoount

In [ ]:
def login():
    sleep(2)
    login_form = driver.find_element(By.ID, 'customer_login')
    fields = login_form.find_elements(By.CLASS_NAME, 'field')
    try:
        fields[0].find_element(By.TAG_NAME, 'input').send_keys(USER_NAME)
        sleep(3)
    except ElementClickInterceptedException:
        close_add()
        sleep(1)
        driver.switch_to.default_content()
        fields[0].find_element(By.TAG_NAME, 'input').send_keys(USER_NAME)
    try:
        fields[1].find_element(By.TAG_NAME, 'input').send_keys(PASSWORD)
        sleep(2)
    except ElementClickInterceptedException:
        close_add()
        sleep(1)
        driver.switch_to.default_content()
        fields[1].find_element(By.TAG_NAME, 'input').send_keys(PASSWORD)
    button = driver.find_element(By.CLASS_NAME, 'button--full-width')
    try:
        button.click()
    except ElementClickInterceptedException:
        close_add()
        sleep(1)
        driver.switch_to.default_content()
        button.click()

## Bypass Recaptcha

In [ ]:
# Not Working - Have to Purchase the API Key and configure into 2nd file
def bypass():
    sleep(3)
    label = driver.find_element(By.XPATH, '//*[@id="g-recaptcha"]')
    label.click()
    sleep(5)
    try:
        submit = driver.find_element(By.XPATH, '//*[@id="MainContent"]/div/form/input[2]')
        submit.click()
    except:
        audio = driver.find_element(By.XPATH, '//*[@id="recaptcha-audio-button"]')
        audio.click()
        link_file = driver.find_element(By.XPATH, '//*[@id="rc-audio"]/div[7]/a').get_attribute('href')
        driver.get(link_file)
        audio_src = driver.find_element(By.TAG_NAME, 'source').get_attribute('src')
        r = sr.Recognizer()
        with sr.AudioFile(audio_src) as source:
            audio_data = r.record(source)
        text = ""
        try:
            text = r.recognize_google(audio_data)
            
            windows = driver.window_handles
            driver.switch_to.window(windows[1])
            
            input_voice = driver.find_element(By.XPATH, '//*[@id="audio-response"]')
            input_voice.send_keys(text)
            
            verify = driver.find_element(By.XPATH, '//*[@id="recaptcha-verify-button"]')
            verify.click()
            sleep(2)
            
            submit = driver.find_element(By.XPATH, '//*[@id="MainContent"]/div/form/input[2]')
            submit.click()
        except sr.UnknownValueError:
            print("Could not transcribe audio")
            exit()

## Search for a product

In [ ]:
def extract_int(text):
    s = ''
    for char in text:
        if char.isdigit():
            s += char
    return int(s)

def extract_pages(default=False):
    tags = driver.find_elements(By.TAG_NAME, 'li')
    button = tags[-17].find_element(By.TAG_NAME, 'a').text
    if default:
        driver.get(tags[-16].get_attribute('href'))
        return
    else:
        return extract_int(button)

def fetch_link(page):
    driver.get(page)
    lists = driver.find_element(By.ID, 'product-grid').find_elements(By.TAG_NAME, 'li')
    for i in range(len(lists)):
        try:
            anchor = lists[i].find_element(By.CLASS_NAME, 'full-unstyled-link')
            link = anchor.get_attribute('href')
            place_order(link)
            driver.get(page)
            lists = driver.find_element(By.ID, 'product-grid').find_elements(By.TAG_NAME, 'li')
        except ElementClickInterceptedException:
            close_add()
            driver.switch_to.default_content()
            fetch_link(page)

def extract_products(MENS_SNEAKER, WOMENS_SNEAKER):
    sneakers = [MENS_SNEAKER, WOMENS_SNEAKER]
    driver.get(MENS_SNEAKER+'1')
    men_wear_pages = extract_pages()
    driver.get(WOMENS_SNEAKER+'1')
    women_wear_pages = extract_pages()
    for i in range(1, men_wear_pages+1):
        driver.get(MENS_SNEAKER+f'{i}')
        fetch_link(MENS_SNEAKER+f'{i}')
    
    for k in range(1, women_wear_pages+1):
        driver.get(WOMENS_SNEAKER+f'{k}')
        fetch_link()

# Place the Order

In [ ]:
def close_add():
    sleep(1)
    link = driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="WigzoPopUpWrapper"]/a').get_attribute('href'))
    driver.get(link)

def extract_labels():
    fieldset = driver.find_element(By.XPATH, '//*[@id="ProductInfo-template--16732352872699__main"]/variant-radios/fieldset')
    labels = fieldset.find_elements(By.TAG_NAME, 'label')
    return labels

def add_to_cart():
    try:
        product_form = driver.find_element(By.TAG_NAME, 'product-form').find_element(By.TAG_NAME, 'form')
        add_to_cart = product_form.find_element(By.CLASS_NAME, 'product-form__buttons').find_element(By.TAG_NAME, 'button')
        add_to_cart.click()
    except ElementClickInterceptedException:
        close_add()
        driver.switch_to.default_content()
        add_to_cart()

def check_out():
    try:
        input_label = driver.find_element(By.XPATH, '//*[@id="Drawer-quantity-1"]')
        input_label.clear()
        input_label.send_keys(randint(1, 2))
        button = driver.find_element(By.XPATH, '//*[@id="CartDrawer-Checkout"]')
        button.click()
    except:
        close_add()
        driver.switch_to.default_content()
        check_out()

def shipping():
    try:
        footer = driver.find_element(By.TAG_NAME, 'form').find_element(By.CLASS_NAME, 'step__footer')
        button = footer.find_element(By.TAG_NAME, 'button')
        button.click()
    except ElementClickInterceptedException:
        close_add()
        driver.switch_to.default_content()
        shipping()
    except:
        pass

def place_order(link):
    driver.get(link)
    labels = extract_labels()
    for i in range(len(labels)):
        labels[i].click()
        sleep(5)
        add_to_cart()
        sleep(3)
        try:
            check_out()
            sleep(2)
            shipping()
            print('Your Order has been placed')
        except StaleElementReferenceException:
            pass
        driver.get(link)
        labels = extract_labels()

## Start the Bot

In [ ]:
if __name__ == "__main__":
    USER_NAME = 'abc@gmail.com'
    PASSWORD = '**********'
    login()
    input('Are you login')
    MENS_SNEAKER = 'https://www.superkicks.in/collections/men-sneakers?filter.v.availability=1&page='
    WOMENS_SNEAKER = 'https://www.superkicks.in/collections/women-sneakers?filter.v.availability=1&page='
    extract_products(MENS_SNEAKER, WOMENS_SNEAKER)